In [2]:
# import numpy as np
# import pandas as pd
# !pip install geopy
# !pip install folium
# !pip install BeautifulSoup4
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# import json
# from geopy.geocoders import Nominatim
# import requests # library to handle requests
# from pandas.io.json import json_normalize
# import matplotlib.cm as cm
# import matplotlib.colors as colors
# from sklearn.cluster import KMeans
# from bs4 import BeautifulSoup
# import xml
# import folium # map rendering library
# r = requests.get('https://opendata.arcgis.com/datasets/823d86e17a6d47808c6e4f1c2dd97928_0.geojson')
# fredericton_geo = r.json()
# neighborhoods_data = fredericton_geo['features']
# g = requests.get('https://opendata.arcgis.com/datasets/6179d35eacb144a5b5fdcc869f86dfb5_0.geojson')
# demog_geo = g.json()
# demog_data = demog_geo['features']
!pip install xlrd
opencrime = 'Crime_by_neighbourhood_2017.xlsx'
workbook = pd.ExcelFile(opencrime)
crime_df = workbook.parse('Crime_by_neighbourhood_2017')
crime_df.head()

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 103 kB 3.7 MB/s eta 0:00:01


XLRDError: No sheet named <'Crime_by_neighbourhood_2017'>

In [ ]:
crime_df.drop(['From_Date', 'To_Date'], axis=1,inplace=True)

# What is the crime count by neighbourhood?

In [ ]:
crime_data = crime_df.groupby(['Neighbourhood']).size().to_frame(name='Count').reset_index()
crime_data.rename(index=str, columns={'Neighbourhood':'Neighbourh','Count':'Crime_Count'}, inplace=True)
crime_data.rename({'Platt': 'Plat'},inplace=True)
crime_data.rename(index=str, columns={'Neighbourhood':'Neighbourh','Count':'Crime_Count'}, inplace=True)
crime_data

In [ ]:
address = 'Fredericton, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Fredericton, New Brunswick is {}, {}.'.format(latitude, longitude))
world_geo = r'world_countries.json' # geojson file
fredericton_1_map = folium.Map(location=[45.97, -66.65], width=1000, height=750,zoom_start=12)
fredericton_1_map

In [ ]:

fredericton_geo = r.json()

threshold_scale = np.linspace(crime_data['Crime_Count'].min(),crime_data['Crime_Count'].max(), 6,dtype=int)
threshold_scale = threshold_scale.tolist()
threshold_scale[-1] = threshold_scale[-1]+1

fredericton_1_map.choropleth(geo_data=fredericton_geo, data=crime_data,columns=['Neighbourh', 'Crime_Count'],
    key_on='feature.properties.Neighbourh', threshold_scale=threshold_scale,fill_color='YlOrRd', fill_opacity=0.7, 
    line_opacity=0.1, legend_name='Fredericton Neighbourhoods')

fredericton_1_map

### Examine Crime Types

In [ ]:
crimetype_data = crime_df.groupby(['Crime_Type']).size().to_frame(name='Count').reset_index()
crimepivot = crime_df.pivot_table(index='Neighbourhood', columns='Crime_Type', aggfunc=pd.Series.count, fill_value=0)
crimepivot

In [ ]:
crimetype_data.plot(x='Crime_Type', y='Count', kind='barh')

### Let's examine theft from vehicles

In [ ]:
mvcrime_df = crime_df.loc[crime_df['Crime_Type'] == 'THEFT FROM MV < $5000']
mvcrime_data = mvcrime_df.groupby(['Neighbourhood']).size().to_frame(name='Count').reset_index()
mvcrime_data.rename({'Platt': 'Plat'},inplace=True)
mvcrime_data.rename(index=str, columns={'Neighbourhood':'Neighbourh','Count':'MVCrime_Count'}, inplace=True)
world_geo = r'world_countries.json' # geojson file
fredericton_c_map = folium.Map(location=[45.91, -66.65], width=1000, height=750,zoom_start=12)
fredericton_c_map

In [ ]:
fredericton_geo = r.json()
threshold_scale = np.linspace(mvcrime_data['MVCrime_Count'].min(), mvcrime_data['MVCrime_Count'].max(),6,dtype=int)
threshold_scale = threshold_scale.tolist()
threshold_scale[-1] = threshold_scale[-1]+1

fredericton_c_map.choropleth(geo_data=fredericton_geo,data=mvcrime_data,columns=['Neighbourh', 'MVCrime_Count'],key_on='feature.properties.Neighbourh',
    threshold_scale=threshold_scale, fill_color='YlOrRd',fill_opacity=0.7,line_opacity=0.1,legend_name='Fredericton Neighbourhoods')
fredericton_c_map